# Librerías

In [82]:
import numpy as np
import pandas as pd
import os
import pydicom
from PIL import Image

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

# Lectura del dataset 

In [51]:
df = pd.read_csv("../data/stage_2_train_labels.csv")
df.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


Comprobaciones sobre el número de registros y si hay elementos duplicados

In [52]:
num_filas = len(df)
num_patientId_unicos = len(np.unique(df["patientId"]))
num_duplicados = num_filas - num_patientId_unicos

print("El número de registros es: " + str(num_filas))
print("El número de patientId únicos es: " + str(num_patientId_unicos))
print("El número de duplicados es: " + str(num_duplicados))


El número de registros es: 30227
El número de patientId únicos es: 26684
El número de duplicados es: 3543


Vamos a identificar esos duplicados y eliminarlos

In [53]:
df_duplicados = df[df.duplicated(["patientId"])]
df_duplicados.head()

,patientId,x,y,width,height,Target
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1
15,00aecb01-a116-45a2-956c-08d2fa55433f,547.0,299.0,119.0,165.0,1
17,00c0b293-48e7-4e16-ac76-9269ba535a62,650.0,511.0,206.0,284.0,1
20,00f08de1-517e-4652-a04f-d1dc9ee48593,571.0,275.0,230.0,476.0,1


In [54]:
df[df["patientId"] == "00704310-78a8-4b38-8475-49f4573b2dbb"]

,patientId,x,y,width,height,Target
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1


Como podemos comprobar hay registros que están duplicados para una misma imagen, esto se entiende a priori que está bien porque un registro indica que hay neumonía en un pulmón, y el otro registro que también hay pulmonía en el otro.

En nuestro caso, tenemos un imagen que queremos clasificar si el target es 1 o 0, pero nos da igual si hay una neumonía en un pulmón, en los dos o en ninguno, es decir, solo nos importa el `patientId` (indica el paciente y la imagen correspondiente) y la variable objetivo `Target` (1 indica que hay neumonía y 0 en caso contrario).

Por lo tanto, vamos a quedarnos con el primer elemnto duplicado, el segundo lo eliminamos del dataset.

In [55]:
df = df.drop_duplicates(subset="patientId")
df.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [56]:
num_filas = len(df)
num_patientId_unicos = len(np.unique(df["patientId"]))
num_duplicados = num_filas - num_patientId_unicos

print("El número de registros es: " + str(num_filas))
print("El número de patientId únicos es: " + str(num_patientId_unicos))
print("El número de duplicados es: " + str(num_duplicados))

El número de registros es: 26684
El número de patientId únicos es: 26684
El número de duplicados es: 0


El siguiente paso es eliminar las columnas innecesasrias, es decir, solo nos hace falta el `patientId` para saber qué paciente es y qué imagen hay que usar, y la variable `Target` para saber si en la imagen hay neumonía o no.

In [57]:
df = df.iloc[:, [0,5]]
print("El número de registros es: " + str(len(df)))
df.head()

El número de registros es: 26684


,patientId,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,0
4,00436515-870c-4b36-a041-de91049b9ab4,1


# Split conjunto de train, test y validation

In [72]:
X_train, X_test, y_train, y_test = train_test_split(df["patientId"], df["Target"], test_size=3000)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=3000)

In [73]:
print("Las dimensiones de X_train son: " + str(X_train.shape))
print("Las dimensiones de y_train son: " + str(y_train.shape))
print("Las dimensiones de X_validation son: " + str(X_validation.shape))
print("Las dimensiones de y_validation son: " + str(y_validation.shape))
print("Las dimensiones de X_test sonn: " + str(X_test.shape))
print("Las dimensiones de y_test sonn: " + str(y_test.shape))

Las dimensiones de X_train son: (20684,)
Las dimensiones de y_train son: (20684,)
Las dimensiones de X_validation son: (3000,)
Las dimensiones de y_validation son: (3000,)
Las dimensiones de X_test sonn: (3000,)
Las dimensiones de y_test sonn: (3000,)


In [78]:
# np.count_nonzero(y_train == 1)

# Transformar las imágenes de dicom a png

En este apartado se va a transformar la imágenes de dicom a png, básicamente para que no comprima la imagen y sea más fácil de trabajar con ella.

La estructura de carpetas que se van a generar son las siguientes:
* train_images: contiene las imágenes del conjunto de train.
* validation_images: contiene las imágenes del conjunto de validation.
* test_images: contiene las imágenes del conjunto de test.

Lo primero de todo es crear las carpeta correspondientes:

In [79]:
# Creación de los directorios
os.mkdir("../data/preprocessed_dataset/train_images")
os.mkdir("../data/preprocessed_dataset/validation_images")
os.mkdir("../data/preprocessed_dataset/test_images")

Lo siguiente es definir una función para que vaya recorriendo la imágenes, las transforme y las almacene en el directorio correspondiente:

In [ ]:
def transform_dcm_png(images, mode="train"):
    path = ""
    root = "../data/stage_2_train_images/"
    
    if mode == "train":
        path = "../data/preprocessed_dataset/train_images"
    if mode == "validation":
        path =  "../data/preprocessed_dataset/validation_images"
    if mode == "test":
        path = "../data/preprocessed_dataset/test_images"
    
    # Empieza la transformación
    for image in images:
        img = pydicom.dcmread(os.path.join(root, image + ".dcm")).pixel_array
        img_save = Image.fromarray(img)
        img_save.save(os.path.join(path, image + ".png"))
        
transform_dcm_png(images=X_train.values, mode="train")
transform_dcm_png(images=X_validation.values, mode="validation")
transform_dcm_png(images=X_test.values, mode="test")

In [88]:
def check_transform(images, mode="train"):
    files = []
    
    if mode == "train":
        files = os.listdir("../data/preprocessed_dataset/train_images")
    if mode == "validation":
        files = os.listdir("../data/preprocessed_dataset/validation_images")
    if mode == "test":
        path = os.listdir("../data/preprocessed_dataset/test_images")
        
    for image in images:
        if not image + ".png" in files:
            return False
    return True

print("Train: " + str(check_transform(images=X_train.values, mode="train")))
print("Validation: " + str(check_transform(images=X_validation.values, mode="validation")))
print("Test: " + str(check_transform(images=X_test.values, mode="test")))

array(['4436d992-c780-474a-b753-4effe7e4deaa',
       '823a9297-1c70-4f5d-8f10-b33205761ca2',
       'ca5c6136-98de-4885-829a-ac8dc00cd00c', ...,
       '0cc27255-7f78-4093-84d7-04190a7d72ec',
       'c8e36af3-6b75-49ac-b144-975daf5aaa38',
       'a2d15397-0a67-4f00-a6ff-e44bf5de8b9c'], dtype=object)

In [ ]:
aux = os.listdir("../data/preprocessed_dataset/train_images")
if '4436d992-c780-474a-b753-4effe7e4deaa.png' in aux:
    print("a")